# Planetary Nebula <a class="tocSkip">
    
This notebook is used to test and showcase the results of my first project. We use spectroscopic data from the [Multi Unit Spectroscopic Explorer](https://www.eso.org/sci/facilities/develop/instruments/muse.html) (MUSE) that has been observed as part of the [PHANGS](https://sites.google.com/view/phangs/home) collaboration.
    
## Basic Packages
    
First we load a bunch of common packages that are used across the project. More specific packages that are only used in one section are loaded later to make it clear where they belong to (this also applies to all custom moduls that were written for this project).

In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

# some basic packages
import os                 # filesystem related stuff
from pathlib import Path  # use instead of os.path and glob
import sys                # system related stuff (e.g. exit)

import errno      # more detailed error messages
import warnings   # handles warnings
import logging    # use logging instead of print

from collections import OrderedDict  

# packages for scientific computing
import numpy as np
import scipy as sp

# packages for creating plots and figures
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# special functions for astronomy 
from astropy.table import Table  # useful datastructure
from astropy.table import vstack # combine multiple tables

from astropy.io import fits      # open fits files
from astropy.io import ascii     # handle normal files

from astropy.wcs import WCS               # handle coordinates
from astropy.coordinates import SkyCoord  # convert pixel to sky coordinates

from astropy.stats import sigma_clipped_stats  # calcualte statistics of images

import astropy.units as u        # handle units

In [ ]:
# set up the standard logger and set verbatim (DEBUG,INFO,WARNING,ERROR,CRITICAL)
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

logging.basicConfig(filename='info.log',
                    filemode='a',
                    format='%(levelname)s %(name)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.DEBUG)

## Reference files

To test the detection routine we compare our results to those from Kreckel et al. (2017)

**Requires** (both already loaded with standard packages)
 * `astropy.io.ascii`
 * `astropy.coordinates.SkyCoord`

In [ ]:
pn_kreckel = ascii.read(os.path.join('..','data','external','kreckel_pn_2017.txt'))

def string_to_ra(string):
    '''convert coordinates from Kreckel et al. (2017) to astropy
    
    the right ascension in the paper is given as 
    "01:36:42.212" but astropy requires "01h36m42.212s".
    This function replaces the ":" with the appropriate character.
    '''
    return string.replace(':','h',1).replace(':','m') + 's'

def string_to_dec(string):
    '''convert coordinates from Kreckel et al. (2017) to astropy
    
    the declination in the paper is given as "01:36:42.212" 
    but astropy requires "01d36m42.212s".
    This function replaces the ":" with the appropriate character.
    '''
    return string.replace(':','d',1).replace(':','m') + 's'

# convert string to astronomical coordinates
pn_kreckel['RA'] = list(map(string_to_ra,pn_kreckel['RA']))
pn_kreckel['DEC'] = list(map(string_to_dec,pn_kreckel['DEC']))
pn_kreckel['SkyCoord'] = SkyCoord(pn_kreckel['RA'],pn_kreckel['DEC'])

# select some subsets (PN from Hermann et al. 2008 or bright sources only)
pn_herrmann = pn_kreckel[[True if i.endswith('a') else False for i in pn_kreckel['ID']]]
pn_bright = pn_kreckel[pn_kreckel['mOIII']<27]

## Read in data

### Define path to raw data

In [ ]:
data_folder = os.path.join('d:',os.sep,'downloads','MUSEDAP')
galaxies = os.listdir(data_folder) 
print('\n'.join(galaxies))

### Read single Galaxy

this uses the `MUSEDAP` class from the `pymuse.io` module 

In [ ]:
from pymuse.io import MUSEDAP

In [ ]:
NGC628 = MUSEDAP('NGC628')

# we also read some additional data from another fits file 
file = os.path.join(data_folder,'NGC628','NGC628_oiii_flux.fits')
with fits.open(file) as hdul:
    setattr(NGC628,'OIII5006_bkg',hdul[0].data)
    
print(NGC628)

## Source Detection

### Based on IRAFStarFinder or DAPStarFinder

the following function is based on this tutorial 

https://photutils.readthedocs.io/en/stable/detection.html

https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder

In [ ]:
from photutils import DAOStarFinder            # DAOFIND routine to detect sources
from photutils import IRAFStarFinder           # IRAF starfind routine to detect star

In [ ]:
from pymuse.detect import detect_sources

In [ ]:
sources = detect_sources(NGC628,'OIII5006',StarFinder=IRAFStarFinder,threshold=3,arcsec_to_pixel=0.2)

#### Compare to Kreckel et al. 2017

In [ ]:
from astropy.coordinates import match_coordinates_sky # match sources against existing catalog
from astropy.coordinates import Angle                 # work with angles (e.g. 1°2′3″)

In [ ]:
ID, angle, Quantity  = match_coordinates_sky(pn_bright['SkyCoord'],sources['SkyCoord'])
within_1_arcsec = len(angle[angle.__lt__(Angle("0.5s"))])

print(f'{within_1_arcsec} of {len(angle)} match within 0.5": {within_1_arcsec / len(angle)*100:.1f} %')
print(f'mean seperation is {angle.mean().to_string(u.arcsec,decimal=True)}"')

#### Plot detected sources

In [ ]:
from pymuse.plot import plot_sources

In [ ]:
position = np.transpose((sources['x'], sources['y']))
positions_kk = np.transpose(pn_bright['SkyCoord'].to_pixel(wcs=NGC628.wcs))
positions = (position,positions_kk)

file = Path.cwd() / '..' / 'reports' / 'figures' / 'sources_IRAF.pdf'

plot_sources(data=NGC628.OIII5006,wcs=NGC628.wcs,positions=positions,filename=file)

#### Cut out detected stars

In [ ]:
from astropy.visualization import simple_norm
from astropy.nddata import Cutout2D
import random

In [ ]:
file = Path.cwd() / '..' / 'reports' / 'figures' / 'stars.pdf'

def plot_detected_stars(self,nrows=10,ncols=10):
    '''create cutouts of the detected sources and plot them
    
    
    '''
    
    # the data we need
    peaks_tbl = self.peaks_tbl
    data      = self.OIII5006
    wcs       = self.wcs
    
    # exclude stars that are too close to the border
    size = 16
    hsize = (size - 1) / 2
    x = peaks_tbl['x']  
    y = peaks_tbl['y']  
    mask = ((x > hsize) & (x < (data.shape[1] -1 - hsize)) &
           (y > hsize) & (y < (data.shape[0] -1 - hsize)))  

    stars_tbl = Table()
    stars_tbl['x'] = x[mask]  
    stars_tbl['y'] = y[mask]  

    # extract_stars does not include wcs information
    #nddata = NDData(data=data,wcs=self.wcs)  
    #stars = extract_stars(nddata, stars_tbl, size=size)  

    # defien the size of the cutout region
    size = u.Quantity((size, size), u.pixel)

    stars = []
    for row in stars_tbl:
        stars.append(Cutout2D(data, (row['x'],row['y']), size,wcs=wcs))
    
    #fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 20),squeeze=True)
    #ax = ax.ravel()
    
    fig = plt.figure(figsize=(100,100))
    
    # get an index idx from 0 to nrows*ncols and a random index i from 0 to len(stars)
    for idx,i in enumerate(random.sample(range(len(stars)), nrows*ncols)):
        ax = fig.add_subplot(nrows,ncols,idx+1,projection=stars[i].wcs)
        
        norm = simple_norm(stars[i].data, 'log', percent=99.)
        ax.imshow(stars[i].data, norm=norm, origin='lower', cmap='viridis')

        
    plt.savefig(file)
    return stars

stars = plot_detected_stars(NGC628)


### Using SExtractor

there is no Python implementation of SExtractor. Instead we run it from the command line

```
sextractor file.fits -c default.sex
```

this will produce a file `test.cat` which contains the position of the sources. We read this table and calculate the sky position wiht astropy

In [ ]:
file = Path.cwd() / '..' / 'data' / 'interim' / 'test.cat'


table = ascii.read(file)
table['SkyCoord'] = SkyCoord.from_pixel(table['X_IMAGE'],table['Y_IMAGE'],NGC628.wcs)

print(f'{len(table)} sources found')

In [ ]:
sources = Table()
sources['x'] = table['X_IMAGE']
sources['y'] = table['Y_IMAGE']
sources['SkyCoord'] = table['SkyCoord']
sources['fwhm'] = 0.8
NGC628.sources = sources

#### Match with known sources

In [ ]:
ID, angle, Quantity  = match_coordinates_sky(pn_bright['SkyCoord'],table['SkyCoord'])
within_1_arcsec = len(angle[angle.__lt__(Angle("0.5s"))])

print(f'{within_1_arcsec} of {len(angle)} match within 0.5": {within_1_arcsec / len(angle)*100:.1f} %')
print(f'mean seperation is {angle.mean().to_string(u.arcsec,decimal=True)}"')
#print(f'mean angle: {angle.mean():.2f}')

#### Plot detected sources

this requires the previously loaded `plot_sources` from `pymuse.plot`

In [ ]:
file = Path.cwd() / '..' / 'reports' / 'figures' / 'sources_sextractor.pdf'

position = np.transpose((sources['x'], sources['y']))
references = np.transpose(pn_bright['SkyCoord'].to_pixel(wcs=NGC628.wcs))
positions = (position,references)

plot_sources(data=NGC628.OIII5006,wcs=NGC628.wcs,positions=positions,filename=file)

### Old line maps

In [ ]:
from photutils import find_peaks

In [ ]:
data_folder = os.path.join('d',os.sep,'downloads','MUSEDAP')
with fits.open(os.path.join(data_folder,'NGC628p','NGC628p_MAPS.fits')) as hdul:
    data = hdul[0].data
    header = hdul[0].header

with fits.open(os.path.join(data_folder,'NGC628p','NGC628p_MAPS_err.fits')) as hdul:
    err = hdul[0].data


In [ ]:
mean, median, std = sigma_clipped_stats(err[~np.isnan(err)], sigma=3.0)

# initialize daofind 
# FWHM is given in arcsec. one pixel is 0.2" 
StarFinder = DAOStarFinder(fwhm=0.8/0.17, 
                           threshold=8.*median,
                           sharplo=0.1, 
                           sharphi=1.0,
                           roundlo=-1,
                           roundhi=1)

# for the source detection we subtract the sigma clipped median
#sources_old = find_peaks(data,12*median,box_size=6)
sources_old = StarFinder(data)

print(f'{len(sources_old):>5.0f}{mean:>8.3f}{median:>8.3f}{std:>8.3f}')

# for consistent table output
for col in sources_old.colnames:
    sources_old[col].info.format = '%.8g'  

sources_old.rename_column('xcentroid','x')
sources_old.rename_column('ycentroid','y')
    
# calculate astronomical coordinates
sources_old['SkyCoord'] = SkyCoord.from_pixel(sources_old['x'],sources_old['y'],WCS(header))
#sources['RaDec'] = sources['SkyCoord'].to_string(style='hmsdms',precision=2)

ID, angle, Quantity  = match_coordinates_sky(pn_bright['SkyCoord'],sources_old['SkyCoord'])
within_1_arcsec = len(angle[angle.__lt__(Angle("0.5s"))])

print(f'{within_1_arcsec} of {len(angle)} match within 0.5": {within_1_arcsec / len(angle)*100:.1f} %')
print(f'mean seperation is {angle.mean().to_string(u.arcsec,decimal=True)}"')

In [ ]:
file = Path.cwd() / '..' / 'reports' / 'figures' / 'sources_old.pdf'

position = np.transpose((sources_old['x'], sources_old['y']))
references = np.transpose(pn_bright['SkyCoord'].to_pixel(wcs=WCS(header)))
positions = (position,references)

plot_sources(data=data,wcs=WCS(header),positions=positions)
                          
plt.xlim([1300,2900])
plt.ylim([1800,3300])

plt.savefig(file)

### Find sources in mock data

In [ ]:
from collections import OrderedDict
from photutils.datasets import (make_random_gaussians_table,
                                make_noise_image,
                                make_gaussian_sources_image)
from photutils import CircularAperture
from astropy.stats import gaussian_sigma_to_fwhm

In [ ]:

def test_detection(StarFinder_Algorithm,sigma_psf,amplitude):
    
    # create mock data
    n_sources = 30
    tshape = (256,256)

    param_ranges = OrderedDict([
                    ('amplitude', [amplitude, amplitude*1.2]),
                    ('x_mean', [0,tshape[0]]),
                    ('y_mean', [0,tshape[1]]),
                    ('x_stddev', [sigma_psf,sigma_psf]),
                    ('y_stddev', [sigma_psf, sigma_psf]),
                    ('theta', [0, 0]) ])

    sources = make_random_gaussians_table(n_sources, param_ranges,
                                          random_state=123)

    image = (make_gaussian_sources_image(tshape, sources) +
             make_noise_image(tshape, type='poisson', mean=6.,
                              random_state=1) +
             make_noise_image(tshape, type='gaussian', mean=0.,
                              stddev=2., random_state=1))

    fwhm = gaussian_sigma_to_fwhm * sigma_psf

    mean, median, std = sigma_clipped_stats(image, sigma=3.0)

    StarFinder = StarFinder_Algorithm(fwhm=fwhm, 
                                      threshold=3.*std,
                                      sharplo=0.1, 
                                      sharphi=1.0,
                                      roundlo=-.2,
                                      roundhi=.2)

    sources_mock = StarFinder(image)

    # for consistent table output
    for col in sources_mock.colnames:
        sources_mock[col].info.format = '%.8g'  

    string = str(StarFinder_Algorithm).split('.')[-1][:-2] + f' sig={sigma_psf} A={amplitude}'
    print(f'{string}: {len(sources_mock):} of {n_sources} sources found')

    positions = np.transpose([sources_mock['xcentroid'],sources_mock['ycentroid']])
    apertures = CircularAperture(positions, r=fwhm)    
    
    return image, apertures, sources, string


nrows = 2
ncols = 4
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 10),
                        squeeze=True)
ax = ax.ravel()


amplitude_lst = [40,200]
sigma_lst = [2.,4.]
StarFinder_lst = [DAOStarFinder,IRAFStarFinder]


settings = []
for f in StarFinder_lst:
    for s in sigma_lst:
        for a in amplitude_lst:
            settings.append((f,s,a))
    
    
for i in range(nrows*ncols):
    f,s,a = settings[i]
    img, ap, sc, string = test_detection(f,s,a)
    
    norm = simple_norm(img, 'log', percent=99.)
    ax[i].imshow(img, norm=norm, origin='lower', cmap='viridis')
    ap.plot(color='red', lw=1., alpha=0.9,ax=ax[i])
    ax[i].scatter(sc['x_mean'],sc['y_mean'],color='red',s=0.8)
    ax[i].set_title(string)
    
plt.show()

## Flux measurement

we use the positions of the previously detected sources to measure the flux of different lines

https://photutils.readthedocs.io/en/stable/aperture.html

the values in the pixels are in units of $10^{-20} \ \mathrm{erg}  \ \mathrm{cm}^{-2} \ \mathrm{s}^{-1} / \mathrm{spaxel}$. To convert into apparent magnitudes we can use

$$
m_{[\mathrm{O\ III}]} = -2.5 \cdot \log F_{[\mathrm{O\ III}]} - 13.74
$$

where $F_{[\mathrm{O\ III}]}$ is given measured in $\mathrm{erg}  \ \mathrm{cm}^{-2} \ \mathrm{s}^{-1}$

$$
\Delta m_{[\mathrm{O\ III}]} = \sqrt{\left(\frac{-2.5 \cdot \Delta F_{[\mathrm{O\ III}]}}{\ln 10 \cdot F_{[\mathrm{O\ III}]}}\right)^2 }
$$

### Percentage of flux in the aperture

for the PSF we assume a 2D gaussian that is centered around the origin and has a variance of $\sigma_x^2 = \sigma_y^2 = \sigma^2$ and an amplitude of $A$
$$
f(x,y) = A \exp\left(-\frac{x^2+y^2}{2\sigma^2}\right)
$$
we can rewrite this in polar coordinates as 
$$
f(r,\phi) = A \exp\left(-\frac{r^2}{2\sigma^2}\right)
$$
this allows us to evaluate the integral
$$
\int_0^{2\pi} \int_0^R f(r,\phi) \mathrm{d} \phi r \mathrm{d} r = 2\pi \sigma^2 A \left(1-\exp \left(-\frac{R^2}{2\sigma^2}\right) \right) 
$$

if $R=x\cdot \mathrm{fwhm} / 2$ (there is a factor of 2 depending on whether the aperture radius or diameter is used) is given in terms of the full width at half maximum, the fraction of light inside the aperture can be written as 

$$
p = 1-\exp(- \ln 2 \cdot x_\mathrm{fwhm}^2)
$$

In [ ]:
x = np.linspace(0.5,4)
p = 1-np.exp(-x**2*np.log(2))
plt.plot(x,100*p)
plt.xlabel('diameter in fwhm')
plt.ylabel('light in aperture in %')
plt.grid()

### Aperture Photometry

In [ ]:
from pymuse.photometry import measure_flux

In [ ]:
flux = measure_flux(NGC628,lines=['OIII5006'],aperture_size=1.5)

#### Compare to Kreckel et al. 2017

we only correct for extinction in the milky way. therefor we use the extinction function from Cardelli, Clayton & Mathis (1989) with $A_V = 0.2$ and $R_V=3.1$. The extinction is calculated with the following package

https://extinction.readthedocs.io/en/latest/

In [ ]:
from extinction import ccm89     # calculate extinction Cardelli et al. (1989)

In [ ]:
ID, angle, Quantity  = match_coordinates_sky(pn_bright['SkyCoord'],sources['SkyCoord'])

pn_bright['mOIII_measured']  = flux[ID]['mOIII']
pn_bright['dmOIII_measured'] = flux[ID]['dmOIII']
pn_bright['sep'] = angle

# calculate extinction correction
extinction = ccm89(wave=np.array([5007.]),a_v=0.2,r_v=3.1,unit='aa')[0]
pn_bright['mOIII_measured'] -= extinction

In [ ]:
fig,ax = plt.subplots(figsize=(7,7))

tolerance = '0.5"'

print(f'{len(pn_bright[angle<Angle(tolerance)])} PN match within {tolerance}')

ax.errorbar(pn_bright[angle<Angle(tolerance)]['mOIII'],
            pn_bright[angle<Angle(tolerance)]['mOIII_measured'],
            yerr=pn_bright[angle<Angle(tolerance)]['dmOIII_measured'],
            fmt='o')

ax.plot([25.5,27.5],[25.5,27.5],color='black',lw=0.4)
ax.set_xlabel(r'$\mathrm{m}_{[\mathrm{OIII}]}$ Kreckel et al. 2017')
ax.set_ylabel(r'$\mathrm{m}_{[\mathrm{OIII}]}$ new',fontsize=16)

plt.show()